In [1]:
import numpy as np
import fastai
from fastai.vision import *
import pandas as pd
import os
from shutil import copyfile
os.listdir("../input")

['resnet101',
 'pytorch-pretrained-image-models',
 'resnet50',
 'aptos2019-blindness-detection',
 'training-kernel',
 'pytorch-pretrained-models']

In [2]:
df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

x_train = df_train['id_code']
y_train = df_train['diagnosis']

In [3]:
os.mkdir("../data")
os.mkdir("../data/train")
for i in range(5):
    os.mkdir("../data/train/"+str(i))
def make_img_folder(x, y):
    for img_name ,diagnosis in zip(x,y):
        if diagnosis == 0:
            copyfile('../input/aptos2019-blindness-detection/train_images/{}.png'.format(img_name), '../data/train/0/{}.png'.format(img_name))
        if diagnosis == 1:
            copyfile('../input/aptos2019-blindness-detection/train_images/{}.png'.format(img_name), '../data/train/1/{}.png'.format(img_name))
        if diagnosis == 2:
            copyfile('../input/aptos2019-blindness-detection/train_images/{}.png'.format(img_name), '../data/train/2/{}.png'.format(img_name))
        if diagnosis == 3:
            copyfile('../input/aptos2019-blindness-detection/train_images/{}.png'.format(img_name), '../data/train/3/{}.png'.format(img_name))
        if diagnosis == 4:
            copyfile('../input/aptos2019-blindness-detection/train_images/{}.png'.format(img_name), '../data/train/4/{}.png'.format(img_name))
            
make_img_folder(x_train, y_train)

In [4]:
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp '../input/pytorch-pretrained-models/resnet152-b121ed2d.pth' '/tmp/.cache/torch/checkpoints/resnet152-b121ed2d.pth'

In [5]:
np.random.seed(42)
src = (ImageList.from_folder("../data/train")
                .split_by_rand_pct(0.2)
                .label_from_folder())
data = (src.transform(tfms=get_transforms(do_flip=False, flip_vert=False, max_rotate=20, max_warp=0), size=224)
           .databunch(bs=64, path='.').normalize(imagenet_stats))

In [6]:
print(data.classes, data.c)

['0', '1', '2', '3', '4'] 5


In [7]:
learn = cnn_learner(data, models.resnet152, metrics=accuracy, model_dir=".", callback_fns=ShowGraph)

In [8]:
learn.load("../input/resnet101/retino_101_128_2");

In [9]:
sample_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [10]:
learn.data.add_test(ImageList.from_df(sample_df,'../input/aptos2019-blindness-detection/',folder='test_images',suffix='.png'))

In [11]:
preds,y = learn.get_preds(DatasetType.Test)

In [12]:
sample_df.diagnosis = preds.argmax(1)
sample_df.head()

,id_code,diagnosis
0,0005cfc8afb6,1
1,003f0afdcd15,2
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,4


In [13]:
sample_df.to_csv('submission.csv',index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 